In [1]:
program es_1
implicit none
integer ::         run_check  ! number of steps
integer, parameter :: runner = 4   ! Numero runner che voglio seguire punto per punto
integer, dimension(:), allocatable :: seed, box
integer :: i, icount1, icount2, icount_rate, ix, irun, istep, sizer, values(1:8)
real, dimension(:), allocatable :: Delta, rnd          ! array numeri random
real, dimension(4) :: P          !array gaussiana con stime
integer, dimension(:), allocatable :: x_N, x2_N ! somma di deviazione e quadrati sulle run 
logical, save :: c = .true.                                                
integer, dimension(:,:), allocatable :: P_N     ! somma sulle run, vogliamo N=8,16,32,64
real, parameter :: pi = 3.14159265359, p_right = 0.5   ! p spostamento a destra
real :: p_left, x_N_th, x2_N_th, sigma2_th, x2_max = 0, dx_max=0, x_max=0




integer, parameter :: N = 64, nruns = 1000





!Genero seed e conservo
call random_seed(sizer)
allocate(seed(sizer))
allocate(box(sizer))       
call date_and_time(values=values)
seed(:) = values(8)

print *,'Here the seed has ',sizer,'components'  
call random_seed(put=seed)
call random_seed(get=box)
open (unit=1,file="seed.dat",status="replace",action="write")
do i = 1 , sizer
   write(unit=1,fmt=*)box(i)
end do
close(1)
print*,'seed data stored in "seed.dat"'





!Chiedo parametri del codice, setto valori iniziali
print *, "Enter number of steps >"
!read *, N
print *, "Enter number of runs >"
!read *, nruns
allocate(rnd(N))
allocate(x_N(N))
allocate(x2_N(N))
allocate(P_N(-N:N,4))
allocate(Delta(nruns))
x_N  = 0
x2_N = 0
P_N  = 0
run_check = nruns/runner
if (run_check == 0)c = .false.

!Setto cose teoriche
p_left = 1- p_right   ! somma p = 1
x_N_th = N*(p_right - p_left)
x2_N_th = (N*(p_right - p_left))**2 + 4*p_right*p_left*N
sigma2_th = 4*p_right*p_left*N






!Eseguo run

open(1,file="es_1_medio.dat",STATUS="REPLACE", ACTION="WRITE")   !File per runner fissato
open(7,file="es_1.dat",STATUS="REPLACE", ACTION="WRITE")         !File per N fissato
write(1,*)"# N=",N,"  nruns=",nruns
write(1,*)""
write(1,*)"# run, i, x_i, x^2_i, media x_i, media x_i^2, sigma^2"

write(7,*)"# N=",N,"  nruns=",nruns
write(7,*)""
write(7,*)"# run, media x_N, media x^2_N, sigma^2, Delta"

do irun = 1, nruns
   ix = 0 ! posizione inziale di ogni run
   call random_number(rnd) ! generi sequenza random numbers
   do istep = 1, N
      if (rnd(istep) > p_left) then !random move
         ix = ix + 1 ! destra
      else
         ix = ix - 1 ! sinistra
      end if
      
      !Istogramma
      if(istep==8)P_N(ix,1) = P_N(ix,1) + 1 ! accumula (solo quando istep = N=8)
      if(istep==16)P_N(ix,2) = P_N(ix,2) + 1 ! accumula (solo quando istep = N=16)
      if(istep==32)P_N(ix,3) = P_N(ix,3) + 1 ! accumula (solo quando istep = N=32)
      if(istep==64)P_N(ix,4) = P_N(ix,4) + 1 ! accumula (solo quando istep = N=64)
      
      
      x_N (istep) = x_N (istep) + ix
      x2_N(istep) = x2_N(istep) + ix**2
      if(c) then     !Fallo solo quando ho num. sufficiente di runner
      !Scrivo per media al variare di N e percorso di ciascun runner
         if(mod(irun,run_check) == 0)write(1,*)irun, istep, ix, ix**2, real(x_N(istep))/irun, &
         real(x2_N(istep))/irun, real(x2_N(istep))/irun-(real(x_N(istep))/irun)**2
      end if
   end do
   if(c) then
      if(mod(irun,run_check) == 0)write(1,*)""   ! GNUplot lavora a blocchi, utile far così
      if(mod(irun,run_check) == 0)write(1,*)""
   end if
   
   
   !Errori
   if(real(x_N(N))/irun>x_max)x_max=real(x_N(N))/irun
   if(real(x2_N(N))/irun>x2_max)x2_max=real(x2_N(N))/irun
   if((real(x2_N(N))/irun - real(x_N(N)**2)/irun) > dx_max)dx_max=real(x2_N(N))/irun - real(x_N(N)**2)/irun     



   !Scrivo per media al variare di run
   Delta(irun) = abs((real(x2_N(N))/irun-(real(x_N(N))/irun)**2)/sigma2_th - 1)
   write(7,*)irun, real(x_N(N))/irun, real(x2_N(N))/irun, real(x2_N(N))/irun-(real(x_N(N))/irun)**2, Delta(irun)
end do



print*,'Intermediate data stored in "es_1_medio.dat"'
print*,'Final data stored in "es_1.dat"'
close(1)
close(7)


print*,"# N=",N,"  nruns=",nruns
print*,"# <x_N> = ",real(x_N(N))/nruns
print*,"# <x_N> error= ",x_max - real(x_N(N))/nruns
print*,"# <x^2_N> = ", real(x2_N(N))/nruns
print*,"# <x^2_N> error= ",x2_max - real(x2_N(N))/nruns
print*,"# <x^2_N> -  <x_N>^2 = ",real(x2_N(N))/nruns - (real(x_N(N))/nruns)**2
print*,"# <x^2_N> -  <x_N>^2 error = ",dx_max - real(x2_N(N))/nruns - (real(x_N(N))/nruns)**2
print*,"# <x_N> th   = ", x_N_th 
print*,"# <x^2_N> th = ", x2_N_th 
print*,"# sigma^2 th = ", sigma2_th
print*,"# Delta_nruns = ", Delta(nruns)

open(7,file="hist_1.dat",STATUS="REPLACE", ACTION="WRITE")
write(7,*)"#N tot =", N, "nruns=",nruns
write(7,*)"# <x_N>   = ",real(x_N(N))/nruns 
write(7,*)"# <x^2_N> = ",real(x2_N(N))/nruns 
write(7,*)"# <x^2_N> - <x_N>^2 = ",real(x2_N(N))/nruns-(real(x_N(N))/nruns)**2 
write(7,*)" "

write(7,*)"#x_i", "frequencies for N=8, 16, 32, 64"
write(7,*)" "

do ix = - N, N
   call gauss_norm(real(x_N(8))/nruns, real(x2_N(8))/nruns-(real(x_N(8))/nruns)**2, ix, P(1))
   call gauss_norm(real(x_N(16))/nruns, real(x2_N(16))/nruns-(real(x_N(16))/nruns)**2, ix, P(2))
   call gauss_norm(real(x_N(32))/nruns, real(x2_N(32))/nruns-(real(x_N(32))/nruns)**2, ix, P(3))
   call gauss_norm(real(x_N(64))/nruns, real(x2_N(64))/nruns-(real(x_N(64))/nruns)**2, ix, P(4))
   write(7,*)ix,(real(P_N(ix,i))/nruns, i=1,4),(P(i), i=1,4)
end do
print*,'P_N data stored in "hist_1.dat"'

close(7)

deallocate (rnd, x_N, x2_N, Delta, P_N, seed, box)
stop


contains

subroutine gauss_norm(mu, var, x, y)
integer, intent(in) :: x
real, intent(in) :: mu, var
real, intent(out) :: y
real, parameter :: pi = 3.14159265359

y = ((2/(pi*var))**0.5)*exp(-(mu - x)**2/(2*var))

end subroutine gauss_norm
end program es_1

 Here the seed has            8 components
 seed data stored in "seed.dat"
 Enter number of steps >
 Enter number of runs >
 Intermediate data stored in "es_1_medio.dat"
 Final data stored in "es_1.dat"
 # N=          64   nruns=        1000
 # <x_N> =    6.40000030E-02
 # <x_N> error=    8.22171402    
 # <x^2_N> =    63.9280014    
 # <x^2_N> error=    105.405327    
 # <x^2_N> -  <x_N>^2 =    63.9239044    
 # <x^2_N> -  <x_N>^2 error =    17.2679005    
 # <x_N> th   =    0.00000000    
 # <x^2_N> th =    64.0000000    
 # sigma^2 th =    64.0000000    
 # Delta_nruns =    1.18899345E-03
 P_N data stored in "hist_1.dat"


Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
